In [10]:
import pandas as pd
import numpy as np
import re
import os
import json
from tqdm import tqdm 
import joblib


In [3]:
forum_folders = os.listdir("./data/dcard_popular")
len(forum_folders)

99

### Preprocess texts of posts

In [9]:
posts = []
for forum in tqdm(forum_folders):
    if forum != ".DS_Store":
        forum_path = f"./data/dcard_popular/{forum}"
        jsonfiles = os.listdir(forum_path)
        for jsonfile in jsonfiles:
            jsonpath = f"{forum_path}/{jsonfile}"
            with open(jsonpath, "r", encoding="utf-8") as f:
                post = json.load(f)
                posts.append(post)
len(posts)

100%|██████████| 99/99 [00:00<00:00, 674.17it/s]


1529

In [12]:
#joblib.dump(posts, "./data/dcard_posts.pkl")

['./data/dcard_posts.pkl']

In [19]:
# Post metadata
for post in posts[:1]:
    print(post.keys())

dict_keys(['id', 'title', 'content', 'excerpt', 'anonymousSchool', 'anonymousDepartment', 'pinned', 'forumId', 'replyId', 'createdAt', 'updatedAt', 'commentCount', 'likeCount', 'tags', 'topics', 'supportedReactions', 'withNickname', 'reportReason', 'hiddenByAuthor', 'meta', 'forumName', 'forumAlias', 'school', 'replyTitle', 'gender', 'personaSubscriptable', 'reactions', 'hidden', 'customStyle', 'isSuspiciousAccount', 'layout', 'withImages', 'withVideos', 'media', 'reportReasonText', 'mediaMeta', 'postAvatar'])


In [23]:
# pick out the columns we want
results = []
for p in posts:
    data = [p['id'],  p['createdAt'], p['forumName'], p['gender'], p['title'], p['content'], p['topics'], p['commentCount'], p['likeCount']]
    results.append(data)

len(results)

1529

In [24]:
# create a dataframe
df = pd.DataFrame(results, columns=['id', 'createdAt', 'forum', 'gender', 'title','content','topics','commentCount','likeCount'])
df.head()

,id,createdAt,forum,gender,title,content,topics,commentCount,likeCount
0,233006165,2020-02-03T04:47:26.058Z,Apple,M,#問題 關於Siri,https://megapx-assets.dcard.tw/images/45dbac63...,"[Apple, 問題, 求救文]",0,3
1,233012413,2020-02-04T02:11:08.007Z,Apple,F,iPhone 11 啊啊啊啊啊啊啊啊😭,#問題\n#求救\n#急在線等\n#HELP \n\n\n本人使用iphone11\n今天無...,"[手機, iPhone]",3,0
2,232795451,2019-12-31T03:40:36.331Z,Apple,M,Apple 板 Mac 教學文章,哈囉大家好\n我是 Apple 板板主 Macintosh \n\n有鑑於版友有問題需要協助...,"[教學, Apple, 問題, macOS]",9,13
3,233010090,2020-02-03T14:59:06.646Z,Apple,M,請問大家都哪裡買五星梅花起子呀？,今天去了光華某家電子材料行\n\n希望買到可以拆macbook pro 的背蓋的梅花起子\n...,"[diy, 起子, 梅花, macbook, 累]",7,0
4,233005085,2020-02-02T20:05:18.803Z,Apple,M,要換iphone11 還是等iphone12,小弟菜味濃，若有錯誤麻煩多多包含😅😅\n小弟目前用8 使用約兩年，操作上都還很順，沒什麼大問...,"[iPhone, 手機, 詢問, 問題, Apple]",33,8


**Tokenization & POS tagging**

In [31]:
trash_re = "@\S+|https?:\S+|http?:\S|[^\u4E00-\u9FD5]"

def clean(texts, trash_re):
    output = []
    for text in texts:
        text = re.sub(trash_re, '', str(text))
        output.append(text)
    return output

In [ ]:
df['content_clean'] = clean(df['content'], trash_re)

In [ ]:
#joblib.dump(df, "./data/dcard_df_cleaned.pkl")

In [34]:
%pip -U ckip-transformers
from ckip_transformers.nlp import CkipWordSegmenter
from ckip_transformers.nlp import CkipPosTagger


Usage:   
  /Library/Developer/CommandLineTools/usr/bin/python3 -m pip <command> [options]

no such option: -U
Note: you may need to restart the kernel to use updated packages.


/Users/cckk2913/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# WS
ws_driver = CkipWordSegmenter(device=0)

In [ ]:
dcard_ws = ws_driver(texts, use_delim = False)
dcard_segmented = [" ".join(ls) for ls in ws]

In [ ]:
#joblib.dump(dcard_ws, "./data/dcard_ws.pkl")
#joblib.dump(dcard_segmented, "./data/dcard_segmented.pkl")

In [ ]:
# POS
pos_driver = CkipPosTagger(device=0)
dcard_pos = pos_driver(dcard_ws, use_delim = False)